In [40]:
#importer les librairies necessaires

import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from matplotlib import pyplot as plt

file_path = './Fero' #Dossier avec les images
colors = vtk.vtkNamedColors() 


In [41]:
#Lire les images 

def loadData():
    reader = vtk.vtkDICOMImageReader() 
    reader.SetDirectoryName(file_path)
    reader.Update()
   
    return reader

In [42]:
#Montrer l'information de l'image

def information(reader, imageData):
    print("Le largeur est :", imageData.GetDimensions()[0]) 
    print("Le longeur est :", imageData.GetDimensions()[1])
    print("L'hauteur est :", imageData.GetDimensions()[2])
    print("La resolution est :", reader.GetPixelSpacing()) #Distance minimal en unites pour separer X,Y,Z
    print("La position du patient est :", reader.GetImagePositionPatient())
    print("La orientation du patient est :",reader.GetImageOrientationPatient())
    print("Le nombre de bits alloués pour chaque pixel dans le fichier est :",reader.GetBitsAllocated())
    print("Le nom du patient est :",reader.GetPatientName())
    
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    #Pour trouver le min et le max d'intensite je dois avoir acces aux valeurs du matrix de l'image
    #car je n'ai pas trouve un commande pour le faire directement avec imageData
    
    #GetDataExtent est necessaire pour avoir un ordre adequat des pixels de l'image
    
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    
    # Load dimensions using `GetDataExtent`
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    # Load spacing values
    ConstPixelSpacing = reader.GetPixelSpacing()
    # Get the 'vtkPointData' object from the 'vtkImageData' object
    pointData = imageData.GetPointData()
    # Ensure that only one array exists within the 'vtkPointData' object
    assert (pointData.GetNumberOfArrays()==1)
    # Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
    arrayData = pointData.GetArray(0)
    # Convert the `vtkArray` to a NumPy array
    ArrayDicom = vtk_to_numpy(arrayData)
    # Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    
    print("Valeur mimimal et maximal d'intensite des pixels en UH :",np.min(ArrayDicom),np.max(ArrayDicom))
    return ArrayDicom

In [49]:
def affichage(reader):
    colors.SetColor("BkgColor", [204, 255, 204, 255])

    # Creer le renderer, la fenetre et l'interactor pour pouvoir avoire une visalisation de l'image et le manipuler
    aRenderer = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(aRenderer)

    iren = vtk.vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)

    # L'isosurface avec un contour value du 300 corresponds au vaisseaux sanguins et coeur du patient.
    # Ce valeur etait trouve grace a l'utilisation de ITK-Snap.
    # On a identifie le stent et les vaisseaux sanguins et on a lu leurs valeurs en UH
    
    visseauxExtractor = vtk.vtkMarchingCubes()
    visseauxExtractor.SetInputConnection(reader.GetOutputPort())
    visseauxExtractor.SetValue(0, 300)
    
    visseauxMapper = vtk.vtkPolyDataMapper()
    visseauxMapper.SetInputConnection(visseauxExtractor.GetOutputPort())
    visseauxMapper.ScalarVisibilityOff()
    
    visseaux = vtk.vtkActor()
    visseaux.SetMapper(visseauxMapper)
    visseaux.GetProperty().SetDiffuseColor(colors.GetColor3d("IndianRed")) #changer le couleur pour ce des vaisseaux sanguins
    visseaux.GetProperty().SetOpacity(0.5) #Changer l'opacite pour voir meilleure le stent
    
    # L'isosurface avec un contour value du 950 corresponds au stent
    stentExtractor = vtk.vtkMarchingCubes()
    stentExtractor.SetInputConnection(reader.GetOutputPort())
    stentExtractor.SetValue(0, 950)

    stentMapper = vtk.vtkPolyDataMapper()
    stentMapper.SetInputConnection(stentExtractor.GetOutputPort())
    stentMapper.ScalarVisibilityOff()

    stent = vtk.vtkActor()
    stent.SetMapper(stentMapper)
    stent.GetProperty().SetDiffuseColor(colors.GetColor3d("Grey")) #changer le couleur pour ce du stent
    stent.GetProperty().SetOpacity(1) #Changer l'opacite pour voir meilleure le stent

    # Creer le camera et une position initial qui permets regarder tout le modele
    aCamera = vtk.vtkCamera()
    aCamera.SetViewUp(0, 0, -1)
    aCamera.SetPosition(0, -1, 0)
    aCamera.SetFocalPoint(0, 0, 0)
    aCamera.ComputeViewPlaneNormal()
    aCamera.Azimuth(30.0)
    aCamera.Elevation(30.0)

    # Ajouter les acteurs au render et mettre en place la camera
    
    aRenderer.AddActor(visseaux)
    aRenderer.AddActor(stent)
    aRenderer.SetActiveCamera(aCamera)
    aRenderer.ResetCamera()
    aCamera.Dolly(1.5)

    # Choisir un bonne couleur de fond qui permets regarder le modele et definir la taille de la fenetre.
    aRenderer.SetBackground(colors.GetColor3d("BkgColor"))
    renWin.SetSize(640, 480)

    aRenderer.ResetCameraClippingRange()

    # Initialiser loop et le commencer.
    iren.Initialize()
    iren.Start()

In [50]:
def main():
    reader=loadData() #Lire les images 
    imageData = vtk.vtkImageData() #Creer une variable pour les imagenes
    imageData.DeepCopy(reader.GetOutput()) #Obtenir matrix 3D avec les donnes de chaque pixel
    ArrayDicom = information(reader, imageData) #Obtenir et imprimer l'information du modele
    affichage(reader) 

In [52]:
main()

Le longeur est : 226
Le largeur est : 128
L'hauteur est : 154
La resolution est : (0.859375, 0.859375, 1.0)
La position du patient est : (-90.5390625, -257.5703125, -276.0)
La orientation du patient est : (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
Le nombre de bits alloués pour chaque pixel dans le fichier est : 16
Le nom du patient est : FEROVIX 
Shape of the array: (226, 128, 154)
Valeur mimimal et maximal d'intensite des pixels en UH : -1080 3622


# Analyse 